### ABC Simple Installation System: Post-processing Upgrade Results
Created on: 10/12/2021 \
By: Lixi Liu (Lixi.Liu@nrel.gov)

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
print(f'Notebook path: {os.getcwd()}')
from pathlib import Path

### Download results online
* unprocessed upgrade results: S3/resbldg-datasets/abc-sis

### Initialize
For modifying plots, can go directly to section 2.3 after initialization

In [ ]:
# local path to downloaded results
iter_path = f'packages_v01' # <-----
result_dir = Path('/Users/lliu2/Documents/ABC Simple Intall Sys/results')
result_path = result_dir / iter_path / 'results'

## create folder for post-processed results:
outdir = result_dir / iter_path / 'processed results'
plot_path = outdir / 'plots'


os.makedirs(outdir, exist_ok=True)
os.makedirs(plot_path, exist_ok=True)

print(f'Run results come from: \n   {result_path}')
print(f'Output directory: \n   {outdir}')

### Functions

In [ ]:
# def get_per_unit_sim_output(df, ref):
#     """
#     ref (df): baseline df
#     """
#     cols = [x for x in df.columns if 
#             x.endswith('_kwh') or
#             x.endswith('_mbtu') or
#             x.endswith('_therm') or
#             x.endswith('_cost_usd') or
#             x.endswith('_ft_2')
#            ]
#     res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
#     df.loc[:, cols] = df.loc[:, cols].replace([None,''],np.nan).divide(
#         res['build_existing_model.units_represented'], axis=0)
    
#     return df
# print('func loaded: "get_per_unit_sim_output"')

# def get_per_unit_sim_output_limited(df, ref):
#     """
#     To reduce computing time
#     ref (df): baseline df
#     """
#     cols = ['simulation_output_report.total_site_natural_gas_therm',
#             'simulation_output_report.total_site_electricity_kwh',
#             'simulation_output_report.total_site_energy_mbtu',
#             'simulation_output_report.upgrade_cost_usd'
#            ]
#     res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
#     df.loc[:, cols] = df.loc[:, cols].replace([None,''],np.nan).divide(
#         res['build_existing_model.units_represented'], axis=0)
    
#     return df
# print('func loaded: "get_per_unit_sim_output_limited"')

def add_sqft_eui(df, ref):
    """
    ARG:
        ref (df): baseline df
    RETURN:
        df with added cols: 'sqft', 'gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_mbtupersqft'
    """
    res = ref.set_index('building_id').reindex(df['building_id']).reset_index()
    df['sqft'] = res['simulation_output_report.floor_area_conditioned_ft_2']
    df['gas_eui_thermpersqft'] = df['simulation_output_report.total_site_natural_gas_therm'].divide(df['sqft']) # therm/sqft
    df['elec_eui_kwhpersqft'] = df['simulation_output_report.total_site_electricity_kwh'].divide(df['sqft']) # kwh/sqft
    df['site_eui_mbtupersqft'] = df['simulation_output_report.total_site_energy_mbtu'].divide(df['sqft']) # mbtu/sqft
    
    for col in ['sqft','gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_mbtupersqft']:
        df.loc[df['simulation_output_report.applicable']==False, col] = np.nan
    
    return df
print('func loaded: "add_sqft_eui"')

def get_res_by_prototype(df, filter_by_df, row):
    """
    ARG:
        df (dataframe): df to slice on
        filter_by_df (dataframe): df used to do the slicing
    RETURN:
        filtered df (dataframe)
    """
    slice_by_df = filter_by_df.copy().set_index('building_id').reindex(df['building_id']).reset_index()
    res_group_i = df[slice_by_df['build_existing_model.geometry_stories'].isin(row['Stories'].split(',')) & \
        slice_by_df['build_existing_model.geometry_wall_type'].isin(row['WallType'].split(',')) & \
        slice_by_df['build_existing_model.vintage_acs'].isin(row['Vintage'].split(',')) & \
        slice_by_df['build_existing_model.geometry_building_type_recs'].isin(['Single-Family Detached'])]
    res_group_i = res_group_i[res_group_i['completed_status']=="Success"]
    
    return res_group_i
print('func loaded: "get_res_by_prototype"')

def load_upgrade(n, file_dir=result_path):
    """
    ARG:
        n (int, str): upgrade number
        file_dir (str): folder in which upgrade can be found, default to main result dir
    RETURN:
        df (dataframe) of upgrade n
    """
    df = pd.read_parquet(os.path.join(file_dir,'upgrades',
                                     f'upgrade={n}/results_up{n:02d}.parquet'))
    return df
print('func loaded: "load_upgrade"')



### 1. BASELINE results

In [ ]:
save_a_copy_in_csv = False # <-----
res = pd.read_csv(result_path / 'results_up00.csv')

# (1) get sqft, gas/elec/site eui
res = add_sqft_eui(res, res)

if save_a_copy_in_csv:
    res.to_csv(os.path.join(combined_res_csv_path,'results_baseline.csv'), index=False)

# (2) get sqft, gas/elec/site eui - redo to get unit-level results
res = add_sqft_eui(res, res)

# check
Nbldg = 111004
Njob = 400

jobs_missing = set(range(1,Njob)) - set(res.job_id.unique())
print(f'- {len(jobs_missing)} jobs missing: {jobs_missing}')
print(f'- {Nbldg-len(res)} buildings ({((Nbldg-len(res))/Nbldg*100):.2f}%)')

res

### 1.1. Check housing charateristics distributions in BASELINE

In [ ]:
proto_chars = ['build_existing_model.geometry_stories',
               'build_existing_model.geometry_wall_type',
               'build_existing_model.vintage_acs',
               'build_existing_model.hvac_cooling_type',
               'build_existing_model.hvac_heating_type_and_fuel',
               'build_existing_model.geometry_floor_area',
               'build_existing_model.census_division',
               'build_existing_model.building_america_climate_zone',
             ]

Ns = len(res.query('completed_status=="Success"')); N = len(res)
print(f'>>> ResStock - {iter_path} - BASELINE result summary:\n')
print(f'  * {Ns} / {N} samples ran successfully, {N-Ns} failed, efficacy: {Ns/N:.1%} \n')

print('>>> Housing characteristics splits:\n')

Res_char = []
for i, char in enumerate(proto_chars,1):
    Nchar = res.groupby(char)['building_id'].count()
    Nchar = Nchar/Ns
    Nchar['N_failed'] = len(res[res[char].isnull()])
    print(f'  * [{i}] {Nchar}\n')
    
    ## append for export
    Nchar = Nchar.rename('fraction').to_frame()
    Nchar['housing_char'] = Nchar.index.name
    Res_char.append(Nchar)
    
Res_char = pd.concat(Res_char, axis=0)
Res_char.index.name = 'sub_char'
Res_char = Res_char.reset_index()
Res_char = Res_char[['housing_char','sub_char','fraction']]

Res_char.to_csv(outdir / 'baseline_housing_char_breakdown.csv', index=False)

## 2B. PACKAGE summary ###

In [ ]:
### upgrade result processing funcs
def assign_utility_rates_to_upgrade(upkg_no, p, ref, HVAC_upgrades_rate_change, for_packages=False):
    
    res = ref.set_index('building_id')[['gas_rate', 'gas_fixed', 'gas_CO2_rate',
                                        'elec_rate', 'elec_fixed', 'elec_CO2_rate'
                                       ]]
    ## assign rates
    p['gas_rate'] = p['building_id'].map(res['gas_rate'])
    p['gas_fixed'] = p['building_id'].map(res['gas_fixed'])
    p['gas_CO2_rate'] = p['building_id'].map(res['gas_CO2_rate'])
    p['elec_rate'] = p['building_id'].map(res['elec_rate'])
    p['elec_fixed'] = p['building_id'].map(res['elec_fixed'])
    p['elec_CO2_rate'] = p['building_id'].map(res['elec_CO2_rate'])
    
    if not for_packages:
        # ind upgrades
        for n in HVAC_upgrades_rate_change.keys():
            new_rates = HVAC_upgrades_rate_change[n]
            p.loc[p['simulation_output_report.applicable']==True, 'gas_rate'] = new_rates[0]
            p.loc[p['simulation_output_report.applicable']==True, 'gas_fixed'] = new_rates[1]
            p.loc[p['simulation_output_report.applicable']==True, 'elec_rate'] = new_rates[2]
            p.loc[p['simulation_output_report.applicable']==True, 'elec_fixed'] = new_rates[3]
    else:
        if isinstance(HVAC_upgrades_rate_change, dict):
            # packages with a dict input
            if upkg_no in HVAC_upgrades_rate_change.keys():
                for m in HVAC_upgrades_rate_change[upkg_no]:
                    idx = (p[(p['apply_upgrade.applicable']==True) &
                           (~p[f'simulation_output_report.option_{m:02d}_cost_usd'].isnull())
                            ]).index
                    p.loc[idx, 'gas_rate'] = NGH_rate
                    p.loc[idx, 'gas_fixed'] = NGH_fixed
                    p.loc[idx, 'elec_rate'] = EH_rate
                    p.loc[idx, 'elec_fixed'] = EH_fixed
        
        else:
            # packages with a list input
            for m in HVAC_upgrades_rate_change:
                idx = (p[(p['apply_upgrade.applicable']==True) &
                       (~p[f'simulation_output_report.option_{m:02d}_cost_usd'].isnull())
                        ]).index
                p.loc[idx, 'gas_rate'] = NGH_rate
                p.loc[idx, 'gas_fixed'] = NGH_fixed
                p.loc[idx, 'elec_rate'] = EH_rate
                p.loc[idx, 'elec_fixed'] = EH_fixed
                
    # assign 0 rates to building with no energy use  
    p.loc[p['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_fixed'] = 0 
    p.loc[p['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_rate'] = 0 
    p.loc[p['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_fixed'] = 0 
    p.loc[p['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_rate'] = 0 
    
    for col in ['gas_rate','gas_fixed','gas_CO2_rate','elec_rate','elec_fixed','elec_CO2_rate']:
        p.loc[p['simulation_output_report.applicable']==False, col] = np.nan

    return p

def combine_upgrade_cost_and_lifetime(p):
    
    ## upgrade costs (sum)
    p['upgrade_cost'] = p['simulation_output_report.upgrade_cost_usd']
    cost_cols = list(x for x in p.columns if x.endswith('cost_usd'))
    p = p.drop(cost_cols, axis=1)
    
    ## upgrade lifetime (min)
    lt_cols = list(x for x in p.columns if x.endswith('lifetime_yrs'))
    p['upgrade_lifetime'] = p[lt_cols].min(axis=1)
    p = p.drop(lt_cols, axis=1)

    p['upgrade_cost'] = p['upgrade_cost'].replace([0, None,''],np.nan)
    p['upgrade_lifetime'] = p['upgrade_lifetime'].replace([0, None,''],np.nan)
    
    return p

def get_annual_totals(pp, get_col_only=False):
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
        
    p['ann_gas_cost'] = \
        p['simulation_output_report.total_site_natural_gas_therm']*p['gas_rate']+p['gas_fixed']
    p['ann_elec_cost'] = \
        p['simulation_output_report.total_site_electricity_kwh']*p['elec_rate']+p['elec_fixed']
    p['ann_energy_cost'] = \
        p['ann_gas_cost'] + p['ann_elec_cost']
    
    p['ann_metric_ton_co2e_gas'] = \
        p['simulation_output_report.total_site_natural_gas_therm']*p['gas_CO2_rate']
    p['ann_metric_ton_co2e_elec'] = \
        p['simulation_output_report.total_site_electricity_kwh']*p['elec_CO2_rate']
    p['ann_metric_ton_co2e'] = \
        p['ann_metric_ton_co2e_gas'] + p['ann_metric_ton_co2e_elec']
    
    if get_col_only:
        return p[['ann_gas_cost','ann_elec_cost','ann_energy_cost',
                 'ann_metric_ton_co2e_gas','ann_metric_ton_co2e_elec','ann_metric_ton_co2e']]
    else:
        return p

def get_annual_gas_elec_site_energy_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_therm_gas_saving'] = ref['simulation_output_report.total_site_natural_gas_therm']-\
         p['simulation_output_report.total_site_natural_gas_therm']
    p['ann_kwh_elec_saving'] = ref['simulation_output_report.total_site_electricity_kwh']-\
         p['simulation_output_report.total_site_electricity_kwh']
    p['ann_mbtu_site_energy_saving'] = ref['simulation_output_report.total_site_energy_mbtu']-\
        p['simulation_output_report.total_site_energy_mbtu']
    
    if get_col_only:
        return p[['ann_therm_gas_saving','ann_kwh_elec_saving','ann_mbtu_site_energy_saving']]
    else:
        return p

def get_annual_energy_cost_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """ 
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_gas_cost_saving'] = \
        ref['simulation_output_report.total_site_natural_gas_therm']*ref['gas_rate']+ref['gas_fixed'] - \
        (p['simulation_output_report.total_site_natural_gas_therm']*p['gas_rate']+p['gas_fixed'])
        
    p['ann_elec_cost_saving'] = \
        ref['simulation_output_report.total_site_electricity_kwh']*ref['elec_rate']+ref['elec_fixed'] - \
        (p['simulation_output_report.total_site_electricity_kwh']*p['elec_rate']+p['elec_fixed'])
    
    p['ann_energy_cost_saving'] = p['ann_gas_cost_saving']+p['ann_elec_cost_saving'] #p[['ann_gas_cost_saving','ann_elec_cost_saving']].sum(axis=1)
    
    if get_col_only:
        return p['ann_energy_cost_saving']
    else:
        return p

def get_annual_metric_ton_co2e_saving(pp, res, get_col_only=False):
    """
    p: upgrade df
    ref: reference scenario df
    get_col_only: whether to return the computed col only or the entire upgrade df p
    """
    if get_col_only:
        p = pp.copy()
    else:
        p = pp
    ref = res.set_index('building_id').reindex(p['building_id']).reset_index()
    
    p['ann_metric_ton_co2e_saving_gas'] = \
        (ref['simulation_output_report.total_site_natural_gas_therm']-\
         p['simulation_output_report.total_site_natural_gas_therm'])*p['gas_CO2_rate']
    
    p['ann_metric_ton_co2e_saving_elec'] = \
        (ref['simulation_output_report.total_site_electricity_kwh']-\
         p['simulation_output_report.total_site_electricity_kwh'])*p['elec_CO2_rate']
    
    p['ann_metric_ton_co2e_saving'] = p['ann_metric_ton_co2e_saving_gas'] + p['ann_metric_ton_co2e_saving_elec']
    
    if get_col_only:
        return p['ann_metric_ton_co2e_saving']
    else:
        return p

def print_metrics_report(p, has_comparative_payback=True):
    ### (1) check for simple_payback > 100 yr or if < 0 yr
    if len(p[~p['simple_payback'].isnull()]):
        print(p['simple_payback'].agg(['min','median','max']))
    spb_100 = p[p['simple_payback']>100]
    if len(spb_100)>0:
        print(f'    *PAYBACK1 - too large* upgrade={n} has {len(spb_100)} simple_payback>100 ' +\
              f'(including {len(p[p["simple_payback"]==np.inf])} INF)')
    spb_neg = p[p['simple_payback']<0]
    if len(spb_neg)>0:
        print(f'    *PAYBACK1 - negative*  upgrade={n} has {len(spb_neg)} simple_payback<0 ' +\
              '(due to negative energy cost saving)')

    ### (2) check for comparative_payback > 100 yr or if < 0 yr
    if has_comparative_payback:
        if len(p[~p['comparative_payback'].isnull()]):
            print(p['comparative_payback'].agg(['min','median','max']))
        spb_100 = p[p['comparative_payback']>100]
        if len(spb_100)>0:
            print(f'    *PAYBACK2 - too large* upgrade={n} has {len(spb_100)} comparative_payback>100 ' +\
                  f'(including {len(p[p["comparative_payback"]==np.inf])} INF)')
        spb_neg = p[p['comparative_payback']<0]
        if len(spb_neg)>0:
            print(f'    *PAYBACK2 - negative*  upgrade={n} has {len(spb_neg)} comparative_payback<0 ' +\
                  '(due to negative energy cost saving)')

    ### (3) check for eui==inf
    for eui in ['pct_delta_gas_eui','pct_delta_elec_eui','pct_delta_site_eui']:
        eui_inf = p[p[eui]==np.inf]
        if len(eui_inf)>0:
            print(f'       *EUI - inf* upgrade={n} has {len(eui_inf)} {eui}=INF ' +\
                  '(due to fuel introduction from upgrade)')

    ### (4) check for neg carbon savings
    ces_neg = p[p['ann_metric_ton_co2e_saving']<0]
    if len(ces_neg)>0:
        print(f'    *CARBON - negative*  upgrade={n} has {len(ces_neg)} carbon saving<0 ')
        
print('funcs loaded')

In [ ]:
# ### set utility rates ###
# NG_rate_multiplier = 1 # <-----

# if NG_rate_multiplier > 1:
#     fn_ext = f'_{NG_rate_multiplier}x_gas_prices' # file name extension to add to relevant results
# else:
#     fn_ext = ''

# ### utility rates ###########################################
# # ref (EIA): 
# # avg ComEd res elec rate 2019: $ 0.1330 /kWh
# # weighted avg IL gas rate 2019: $ 0.77183 /therm

# ## electricity ##
# # annual fixed rates = monthly x 12
# EH_fixed = 15.70 * 12 # annual
# NEH_fixed = 14.28 *12 # annual
# # avg of summer rates (J,J,A,S) and non-summer rates
# EH_rate = (0.10273*4+0.10952*8)/12 # 0.08019, $/kWh, electric rate for electric heating customers
# NEH_rate = (0.12168*4+0.12847*8)/12 # 0.09889, $/kWh, electric rate for non-electric heating customers
# # marginal carbon emission factor:
# elec_CO2_rate = 0.000834702 # metric tons of CO2e/kWh (0.2446 tons/mbtu)

# ## gas ##
# # annual fixed rates = monthly x 12
# GH_fixed = 45.32 * 12
# NGH_fixed = 21.51 * 12
# # variable rates
# GH_rate = 0.61648 * NG_rate_multiplier # 0.19477, $/therm, gas rate for NG heating customers
# NGH_rate = 0.56758 * NG_rate_multiplier # 0.14964, $/therm, gas rate for non-NG heating customers
# # marginal carbon emission factor:
# gas_CO2_rate = 0.00532181 # metric tons of CO2e/therm (0.0532 tons/mbtu)

# ### upgrades that will cause utility rate change: ###########################################
# HVAC_upgrades_rate_change = {
#     17: [4,6,7], # ASHP
#     18: [4,6,7,8], # MSHP,
# }

# # assign rates accordingly
# Elec_heating_types = ['Electricity Baseboard','Electricity ASHP','Electricity Electric Furnace',
#                       'Electricity Electric Boiler', 'Electricity Electric Wall Furnace']
# NG_heating_types = ['Natural Gas Fuel Wall/Floor Furnace', 'Natural Gas Fuel Furnace',
#                     'Natural Gas Fuel Boiler']

# res['gas_rate'] = NGH_rate
# res['gas_fixed'] = NGH_fixed
# res['gas_CO2_rate'] = gas_CO2_rate
# res['elec_rate'] = NEH_rate
# res['elec_fixed'] = NEH_fixed
# res['elec_CO2_rate'] = elec_CO2_rate

# res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(NG_heating_types)]).index,
#        'gas_rate'] = GH_rate
# res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(NG_heating_types)]).index,
#        'gas_fixed'] = GH_fixed
# res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(Elec_heating_types)]).index,
#         'elec_rate'] = EH_rate
# res.loc[(res[res['build_existing_model.hvac_heating_type_and_fuel'].isin(Elec_heating_types)]).index,
#         'elec_fixed'] = EH_fixed

# # assign 0 gas rates to building with no gas use 
# res.loc[res['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_fixed'] = 0 
# res.loc[res['simulation_output_report.total_site_natural_gas_therm'].isin([0, np.nan]), 'gas_rate'] = 0 
# res.loc[res['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_fixed'] = 0 
# res.loc[res['simulation_output_report.total_site_electricity_kwh'].isin([0, np.nan]), 'elec_rate'] = 0

# for col in ['gas_rate','gas_fixed','gas_CO2_rate','elec_rate','elec_fixed','elec_CO2_rate']:
#     res.loc[res['completed_status']!='Success', col] = np.nan

# print(f'Natural gas rate multiplier: {NG_rate_multiplier}')


### Get baseline results

In [ ]:
#### count upgrades
N_upgrades = 10 # <---

print(f'>>> {iter_path} has {N_upgrades:,} packages...')
    
## get summary table
summary_upgrades = []

## initialize dB table
# all_proto_upgrades = []

package_list = range(1,1+N_upgrades)
for n in package_list:
    p = pd.read_csv(result_path / f'results_up{n:02d}.csv')
    print(f'\nPackage {n}')

    ### assign utility rates
#     p = assign_utility_rates_to_upgrade(n, p, res, HVAC_upgrades_rate_change, for_packages=True)
    
    ### collapse upgrade cost and lifetime cols
    p = combine_upgrade_cost_and_lifetime(p)

    ### check if upgrade has 0 successful sims
    if len(p[p['completed_status']=='Success']) == 0:
        print(f' * upgrade={n} has 0 successful simulations')

    ### calculate metrics
    p = add_sqft_eui(p, res)
    EUIi = ['gas_eui_thermpersqft','elec_eui_kwhpersqft','site_eui_mbtupersqft']
    EUIo = ['gas_eui','elec_eui','site_eui']
    for vari, varo in zip(EUIi, EUIo):
        p[f'pct_delta_{varo}'] = ((p[vari]-res[vari])/res[vari]*100)


    # annual energy saving:
    p = get_annual_gas_elec_site_energy_saving(p, res)

    # annual energy cost saving:
#     p = get_annual_energy_cost_saving(p, res)
    
    # annual kBtu saved per upgrade cost:
#     p['ann_kbtu_saved_per_dollar'] = p['ann_mbtu_site_energy_saving'].divide(
#                             p['upgrade_cost'], axis=0)*1000

    # simple payback
#     p['simple_payback'] = p['upgrade_cost']/p['ann_energy_cost_saving']

    # annual metric ton carbon emission savings:
#     p = get_annual_metric_ton_co2e_saving(p, res)

    ### check for neg/large paybacks, inf eui, neg carbon savings 
#     print_metrics_report(p, has_comparative_payback=False)
    
    ### subset to only those that have been applied with the upgrades successfully:
    upgrade_name = p['apply_upgrade.upgrade_name'].replace('',np.nan).dropna(axis=0).unique()[0]
    p = p[p['apply_upgrade.applicable']==True].reset_index(drop=True)
    p['package_no'] = n
    
    ### add to dB
#     all_proto_upgrades.append(p)
    
    for eui in ['pct_delta_gas_eui','pct_delta_elec_eui','pct_delta_site_eui']:
        p[eui] = p[eui].replace([np.inf, -np.inf], np.nan) # for mean calc
        
    ### add to summary table
    summ = pd.DataFrame(index=[0])
    level0 = ''
    summ[(level0, 'upgrade_no')] = n
    summ[(level0, 'upgrade_name')] = upgrade_name
    summ[(level0, 'sample_weight')] = res['build_existing_model.sample_weight'].mean()
    summ[(level0, 'n_applied')] = len(p)
    summ[(level0, 'n_success')] = len(p[p['completed_status']=='Success'])
    summ[(level0, 'n_fail')] = len(p[p['completed_status']=='Fail'])
    summ[(level0, 'n_invalid')] = len(p[p['completed_status']=='Invalid'])
    summ[(level0, 'pct_success')] = round(summ[(level0,'n_success')]/summ[(level0,'n_applied')]*100,3)
    summ.columns = pd.MultiIndex.from_tuples(summ.columns)

    para = ['mean','std', 'min','50%','max']
    p = p[p['completed_status']=='Success'].reset_index(drop=True)
    
    ## (0) Upgrade cost
    summ = pd.concat([
        summ,
        p['upgrade_cost'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    
    ## (1) Baselines
    consum = pd.DataFrame(index=[0])
    level1 = 'baseline - mean'
    # 1.1. Annual Totals
    consum[(level1, 'gas (therm)')] = round(
        res.loc[p.index, 'simulation_output_report.total_site_natural_gas_therm'].mean(), 
        3)
    consum[(level1, 'elec (kwh)')] = round(
        res.loc[p.index, 'simulation_output_report.total_site_electricity_kwh'].mean(), 
        3)
    consum[(level1, 'site (mbtu)')] = round(
        res.loc[p.index, 'simulation_output_report.total_site_energy_mbtu'].mean(), 
        3)
    
    # 1.2. EUIs
    consum[(level1, 'gas_eui (therm/sqft)')] = round(
        res.loc[p.index, 'gas_eui_thermpersqft'].mean(), 
        3)
    consum[(level1, 'elec_eui (kwh/sqft)')] = round(
        res.loc[p.index, 'elec_eui_kwhpersqft'].mean(), 
        3)
    consum[(level1, 'site_eui (mbtu/sqft)')] = round(
        res.loc[p.index, 'site_eui_mbtupersqft'].mean(), 
        3)
    
    ## (2) Upgrades
    level2 = 'upgrade - mean'
    # 2.1. Annual Totals
    consum[(level2, 'gas (therm)')] = round(p['simulation_output_report.total_site_natural_gas_therm'].mean(), 3)
    consum[(level2, 'elec (kwh)')] = round(p['simulation_output_report.total_site_electricity_kwh'].mean(), 3)
    consum[(level2, 'site (mbtu)')] = round(p['simulation_output_report.total_site_energy_mbtu'].mean(), 3)
    
    # 2.2. EUIs
    consum[(level2, 'gas_eui (therm/sqft)')] = round(p['gas_eui_thermpersqft'].mean(), 3)
    consum[(level2, 'elec_eui (kwh/sqft)')] = round(p['elec_eui_kwhpersqft'].mean(), 3)
    consum[(level2, 'site_eui (mbtu/sqft)')] = round(p['site_eui_mbtupersqft'].mean(), 3)
    
    consum.columns = pd.MultiIndex.from_tuples(consum.columns)
    summ = pd.concat([summ, consum], axis=1)
    
    
    ## (3-1) Savings ##
    level3 = 'savings'
    # 3-1.1 Annual Totals
    summ = pd.concat([
        summ,
        p['ann_therm_gas_saving'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    summ = pd.concat([
        summ,
        p['ann_kwh_elec_saving'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    summ = pd.concat([
        summ,
        p['ann_mbtu_site_energy_saving'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    
    summ = pd.concat([
        summ,
        p['pct_delta_gas_eui'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    summ = pd.concat([
        summ,
        p['pct_delta_elec_eui'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    summ = pd.concat([
        summ,
        p['pct_delta_site_eui'].describe()[para].apply(lambda x: round(x, 2)).to_frame().unstack().to_frame().transpose()
    ], axis=1)
    
 
    
#     summ['mean_ann_kbtu_saved_per_dollar'] = round(p['ann_kbtu_saved_per_dollar'].mean(),3) # annual kBtu saved per upgrade cost
#     summ['mean_ann_energy_cost_saving'] = round(p['ann_energy_cost_saving'].mean(),2)
#     summ['mean_ann_metric_ton_co2e_saving'] = round(p['ann_metric_ton_co2e_saving'].mean(),3)

#     summ['median_simple_payback'] = round(p['simple_payback'].median(),3)
    
#     p['simple_payback'] = p['simple_payback'].replace([np.inf, -np.inf], np.nan) # for mean calc
#     summ['pct_pos_simple_payback_actual'] = round(len(p[p['simple_payback']>=0])/len(p)*100, 3)
#     summ['mean_pos_simple_payback_actual'] = round(p.loc[p['simple_payback']>=0, 'simple_payback'].mean(),3)

#     # filter: min 1 cent energy cost savings **
#     min_energy_cost_saving = 0.1 # <----- **
#     summ['pct_pos_simple_payback_filtered'] = round(len(
#         p[(p['simple_payback']>=0) & (p['ann_energy_cost_saving']>=min_energy_cost_saving)]
#     )/len(p)*100, 3)
#     summ['mean_pos_simple_payback_filtered'] = round(
#         p.loc[(p['simple_payback']>=0) & (p['ann_energy_cost_saving']>=min_energy_cost_saving), 
#            'simple_payback'].mean(),3)
    
    summary_upgrades.append(summ)
    
summary_upgrades = pd.concat(summary_upgrades).reset_index(drop=True)
summary_upgrades


In [ ]:
### export UPGRADE summary
summary_upgrades.to_csv(outdir / f'upgrades_summary.csv', index=False)
print(f'UPGRADE summary table saved')